In [ ]:
import codecs
import os
tag_dic = {"disease": "疾病",
           "drug": "药物",
           "surgery": "手术",
           "symptom": "症状",
           "prevention": "预防措施",
           "inspection_method": "观察方法",
           "diagnostic_technology": "诊断技术",
           "non-drug_therapy": "物理疗法",
           "nursing_technology": "护理技术",
           "cause_of_disease": "病因",
           "pathological_mechanism": "病理",
           "medical_indicator": "医疗指标"}
 # 由于brat不支持中文标签 所以tag_dic 将标签转化成中文的
def from_ann2dic(r_ann_path, r_txt_path, w_path):
    q_dic = {}
    print("开始读取文件:%s" % r_ann_path)
    with codecs.open(r_ann_path, "r", encoding="utf-8") as f:#读取标签文件
        line = f.readline()
        line = line.strip("\n\r")
        while line != "":
            line_arr = line.split()
            print(line_arr)
            try:
                cls = tag_dic[line_arr[1]] #line_arr[1]为标签名称 通过tag_dic 查字典后转化为中文标签
                start_index = int(line_arr[2])#标签起始位置
                end_index = int(line_arr[3])#标签的结束位置
                length = end_index - start_index#标签长度
                if length==1:
                    q_dic[start_index] = ("S-%s" % cls)
                else:
                    for r in range(length):#为标签内的字标号
                        if r == 0:#第一个字标B
                            q_dic[start_index] = ("B-%s" % cls) #在字典内添加字标签 eg: 21：B-药物
                        elif r==length-1:
                            q_dic[start_index+r] = ("E-%s" % cls)
                        else:#标签内其余的标I
                            q_dic[start_index + r] = ("I-%s" % cls)#在字典内添加字标签 eg: 22：I-药物
            except:
                line = f.readline()
                line = line.strip("\n\r")
                continue#出现异常则跳过 继续循环
            line = f.readline()
            line = line.strip("\n\r")
            print(q_dic)
    print("开始读取文件:%s" % r_txt_path)
    with codecs.open(r_txt_path, "r", encoding="utf-8") as f:#读取文本文档
        content_str = f.read()
    print("开始写入文本%s" % w_path)#开始将标签写入文本
    with codecs.open(w_path, "w", encoding="utf-8") as w:
        for i, str in enumerate(content_str):
            if str is " " or str == "" or str == "\n":
                print("===============")
            elif str == "\r":
                w.write("\r")
            else:
                if i in q_dic:#文本中的位置若存在标签 赋给tag标签
                    tag = q_dic[i]
                else:
                    tag = "O"  # 大写字母O
                w.write('%s %s\n' % (str, tag))#每个字一行每个字都有标签 分隔符在此设置
                if str=="。"or str=="?"or str=="!":#输入要求 保证每一句话后都有一个空格
                    w.write("\n")
        w.write('%s\n' % "END O")
def rw0(data_root_dir, w_path):#合并文件 生成训练测试集
    if os.path.exists(w_path):
        os.remove(w_path)
    for file in os.listdir(data_root_dir):
        path = os.path.join(data_root_dir, file)
        if file.endswith("abstract_007-new.txt"):
            # 重命名为dev.txt
            os.rename(path, os.path.join(data_root_dir, "dev.txt"))
            continue
        #if file.endswith("abstract_011-new.txt"):
            # 重命名为test.txt
            #os.rename(path, os.path.join(data_root_dir, "test.txt"))
           # continue
        q_list = []
        print("开始读取文件:%s" % file)
        with codecs.open(path, "r", encoding="utf-8") as f:
            line = f.readline()
            line = line.strip("\n\r")
            while line != "END O":
                q_list.append(line)
                line = f.readline()
                line = line.strip("\n\r")
        print("开始写入文本%s" % w_path)
        with codecs.open(w_path, "a", encoding="utf-8") as f:
            for item in q_list:
                f.write('%s\n' % item)

data_dir = "datas"
for file in os.listdir(data_dir):
    if file.find(".") == -1:
        continue
    file_name = file[0:file.find(".")]
    r_ann_path = os.path.join(data_dir, "%s.ann" % file_name)
    r_txt_path = os.path.join(data_dir, "%s.txt" % file_name)
    w_path = "%s/new/%s-new.txt" % (data_dir, file_name)
    from_ann2dic(r_ann_path, r_txt_path, w_path)
    # 生成train.txt、dev.txt、test.txt
rw0("%s/new" % data_dir, "%s/new/train.txt" % data_dir)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [33]:
q_dic 

NameError: name 'q_dic' is not defined

In [29]:
import codecs
import os
class BIOES:
    def __init__(self):
        self.q_dic = {}
        self.content_str=''
        self.tag_dic = {"disease": "疾病",
           "drug": "药物",
           "surgery": "手术",
           "symptom": "症状",
           "prevention": "预防措施",
           "inspection_method": "观察方法",
           "diagnostic_technology": "诊断技术",
           "non-drug_therapy": "物理疗法",
           "nursing_technology": "护理技术",
           "cause_of_disease": "病因",
           "pathological_mechanism": "病理",
           "medical_indicator": "医疗指标"}
    def ann2chara_tag(self,r_ann_path):
        self.q_dic = {}
        with codecs.open(r_ann_path, "r", encoding="utf-8") as f:#读取标签文件
            line = f.readline()
            line = line.strip("\n\r")
            while line != "":
                line_arr = line.split()
                print(line_arr)
                try:
                    cls = self.tag_dic[line_arr[1]] #line_arr[1]为标签名称 通过tag_dic 查字典后转化为中文标签
                    start_index = int(line_arr[2])#标签起始位置
                    end_index = int(line_arr[3])#标签的结束位置
                    length = end_index - start_index#标签长度
                    if length==1:
                        self.q_dic[start_index] = ("S-%s" % cls)
                    else:
                        for r in range(length):#为标签内的字标号
                            if r == 0:#第一个字标B
                                self.q_dic[start_index] = ("B-%s" % cls) #在字典内添加字标签 eg: 21：B-药物
                            elif r==length-1:
                                self.q_dic[start_index+r] = ("E-%s" % cls)
                            else:#标签内其余的标I
                                self.q_dic[start_index + r] = ("I-%s" % cls)#在字典内添加字标签 eg: 22：I-药物
                except:
                    line = f.readline()
                    line = line.strip("\n\r")
                    continue#出现异常则跳过 继续循环
                line = f.readline()
                line = line.strip("\n\r")
    def readtxt(self,r_txt_path):
        print("开始读取文件:%s" % r_txt_path)
        with codecs.open(r_txt_path, "r", encoding="utf-8") as f:#读取文本文档
            self.content_str = f.read()        
    def toBIOES(self,w_path):
        print("开始写入文本%s" % w_path)#开始将标签写入文本
        with codecs.open(w_path, "w", encoding="utf-8") as w:
            for i, str in enumerate(self.content_str):
                if str is " " or str == "" or str == "\n":
                    print("===============")
                elif str == "\r":
                    w.write("\r")
                else:
                    if i in self.q_dic:#文本中的位置若存在标签 赋给tag标签
                        tag = self.q_dic[i]
                    else:
                        tag = "O"  # 大写字母O
                    w.write('%s %s\n' % (str, tag))#每个字一行每个字都有标签 分隔符在此设置
                    if str=="。"or str=="?"or str=="!":#输入要求 保证每一句话后都有一个空格
                        w.write("\n")
            w.write('%s\n' % "END O")
    def combine(self,data_root_dir, w_path):#合并文件 生成训练测试集
        if os.path.exists(w_path):
            os.remove(w_path)
        for file in os.listdir(data_root_dir):
            path = os.path.join(data_root_dir, file)
            if file.endswith("abstract_007-new.txt"):
                # 重命名为dev.txt
                os.rename(path, os.path.join(data_root_dir, "dev.txt"))
                continue
            #if file.endswith("abstract_011-new.txt"):
                # 重命名为test.txt
                #os.rename(path, os.path.join(data_root_dir, "test.txt"))
               # continue
            q_list = []
            print("开始读取文件:%s" % file)
            with codecs.open(path, "r", encoding="utf-8") as f:
                line = f.readline()
                line = line.strip("\n\r")
                while line != "END O":
                    q_list.append(line)
                    line = f.readline()
                    line = line.strip("\n\r")
            print("开始写入文本%s" % w_path)
            with codecs.open(w_path, "a", encoding="utf-8") as f:
                for item in q_list:
                    f.write('%s\n' % item)
    def run(self):
        data_dir = "datas"
        for file in os.listdir(data_dir):
            if file.find(".") == -1:
                continue
            file_name = file[0:file.find(".")]
            r_ann_path = os.path.join(data_dir, "%s.ann" % file_name)
            r_txt_path = os.path.join(data_dir, "%s.txt" % file_name)
            w_path = "%s/new/%s-new.txt" % (data_dir, file_name)
            self.ann2chara_tag(r_ann_path)
            self.readtxt(r_txt_path)
            self.toBIOES(w_path)
            # 生成train.txt、dev.txt、test.txt
        self.combine("%s/new" % data_dir, "%s/new/train.txt" % data_dir)

In [30]:
bioes=BIOES()
bioes.run()

['T1', 'disease', '250', '257', '阿尔茨海默氏病']
['T2', 'symptom', '283', '288', '进行性痴呆']
['T3', 'cause_of_disease', '354', '356', '遗传']
['R1', 'has_clinical_manifestation', 'Arg1:T1', 'Arg2:T2']
['T4', 'disease', '24', '30', '阿尔茨海默病']
['T6', 'drug', '713', '718', '补肾健脾方']
['T7', 'disease', '561', '567', '阿尔茨海默病']
['T8', 'drug', '1520', '1523', '奎硫平']
['T9', 'drug', '1524', '1527', '利培酮']
['T10', 'disease', '1528', '1534', '阿尔茨海默病']
['R5', 'medical_treat', 'Arg1:T8', 'Arg2:T10']
['R6', 'medical_treat', 'Arg1:T9', 'Arg2:T10']
['T11', 'disease', '2565', '2571', '阿尔茨海默病']
['T12', 'pathological_mechanism', '2589', '2597', '大脑皮层厚度萎缩']
['T13', 'symptom', '2598', '2606', '认知功能逐步下降']
['R8', 'has_clinical_manifestation', 'Arg1:T11', 'Arg2:T13']
['T14', 'disease', '4734', '4740', '阿尔兹海默症']
['T15', 'symptom', '4744', '4750', '智力缓慢丧失']
['R9', 'has_clinical_manifestation', 'Arg1:T14', 'Arg2:T15']
['T16', 'drug', '5317', '5320', '都可喜']
['T17', 'drug', '5322', '5327', '补肾益智汤']
['T18', 'disease', '5329', '5

['T1069', 'medical_indicator', '21214', '21222', '-1887A/G']
['T1071', 'medical_indicator', '21223', '21231', '-2315A/G']
['T1070', 'medical_indicator', '21319', '21327', '-1887A/G']
['T1073', 'medical_indicator', '21328', '21336', '-2315A/G']
['T952', 'diagnostic_technology', '16774', '16781', '神经元细胞计数']
['T487', 'cause_of_disease', '1146', '1151', 'D-半乳糖']
['T499', 'cause_of_disease', '4813', '4816', 'Tau']
['T415', 'cause_of_disease', '7006', '7009', '甘丙肽']
['T126', 'cause_of_disease', '15761', '15768', 'Aβ25-35']
['T599', 'cause_of_disease', '15769', '15774', '鹅膏覃氨酸']
['T906', 'cause_of_disease', '19223', '19228', 'D442G']
['T1011', 'medical_indicator', '19489', '19504', '(TGF)-β1-509C/T']
['T1024', 'medical_indicator', '19505', '19510', 'T869C']
['T1029', 'medical_indicator', '19587', '19600', 'TGF-β1-509C/T']
['T1158', 'medical_indicator', '19601', '19606', 'T869C']
['T1036', 'cause_of_disease', '19888', '19901', 'TGF-β1-509C/T']
['T1159', 'cause_of_disease', '19902', '19907', 'T

['T1334', 'diagnostic_technology', '20605', '20608', 'PCR']
['T1335', 'diagnostic_technology', '20611', '20614', 'PCR']
['T1336', 'cause_of_disease', '20663', '20666', 'APP']
['T1065', 'diagnostic_technology', '20916', '20933', '聚合酶链反应-限制性片段长度多态性']
['T1337', 'diagnostic_technology', '20934', '20999', 'polymerase', 'chain', 'reaction-restriction', 'fragmentlength', 'polymorphism']
['T1338', 'diagnostic_technology', '21000', '21008', 'PCR-RFLP']
['T1067', 'diagnostic_technology', '21081', '21083', '分型']
['T1339', 'cause_of_disease', '21055', '21060', '载脂蛋白E']
['T1340', 'cause_of_disease', '21040', '21046', 'GRIN2B']
['T1341', 'cause_of_disease', '21061', '21075', 'apolipoprotein']
['T1342', 'cause_of_disease', '21076', '21080', 'APOE']
['T573', 'inspection_method', '21936', '21942', '汉密尔顿量表']
['T1343', 'inspection_method', '21943', '21947', 'HAMA']
['T696', 'inspection_method', '21953', '21961', '简易智力状态检查']
['T1344', 'inspection_method', '21962', '21966', 'MMSE']
['T749', 'inspection_met

['T1102', 'medical_indicator', '19404', '19431', 'tumor', 'necrosis', 'factor', 'alpha']
['T1103', 'medical_indicator', '19432', '19437', 'TNF-α']
['T959', 'medical_indicator', '19463', '19468', '总记忆错误']
['T1104', 'medical_indicator', '19469', '19471', 'TE']
['T960', 'medical_indicator', '19473', '19479', '参考记忆错误']
['T1105', 'medical_indicator', '19480', '19483', 'RME']
['T961', 'medical_indicator', '19485', '19491', '工作记忆错误']
['T1106', 'medical_indicator', '19492', '19495', 'WME']
['T518', 'diagnostic_technology', '19671', '19674', '磁共振']
['T1107', 'diagnostic_technology', '19677', '19680', 'MRI']
['T971', 'medical_indicator', '19757', '19766', '简易认知状态检查表']
['T1108', 'medical_indicator', '19767', '19772', 'MM', 'SE']
['T1109', 'medical_indicator', '19738', '19743', '小脑总体积']
['T1110', 'medical_indicator', '19745', '19749', '亚区体积']
['T256', 'cause_of_disease', '4746', '4751', '淀粉样蛋白']
['T306', 'pathological_mechanism', '5930', '5934', '细胞凋亡']
['T322', 'pathological_mechanism', '6301', '

['T766', 'disease', '14471', '14477', '阿尔茨海默病']
['T767', 'disease', '14478', '14480', 'AD']
['T768', 'cause_of_disease', '14500', '14505', 'γ-分泌酶']
['T769', 'cause_of_disease', '14510', '14512', 'Aβ']
['T770', 'disease', '14526', '14528', 'AD']
['T772', 'cause_of_disease', '14561', '14566', 'γ-分泌酶']
['T773', 'cause_of_disease', '14575', '14577', 'Aβ']
['T774', 'medical_indicator', '14589', '14591', 'Aβ']
['T775', 'disease', '14596', '14601', '轻中度AD']
['T776', 'disease', '14675', '14693', 'Alzheimer’sdisease']
['T777', 'disease', '14694', '14696', 'AD']
['T779', 'drug', '14728', '14734', 'AChEIs']
['T780', 'disease', '14739', '14741', 'AD']
['T781', 'drug', '14748', '14754', 'AChEIs']
['T782', 'disease', '14766', '14768', 'AD']
['T783', 'disease', '14826', '14828', '痴呆']
['T784', 'disease', '14841', '14847', '阿尔茨海默病']
['T785', 'disease', '14848', '14850', '痴呆']
['T786', 'drug', '14873', '14877', '加兰他敏']
['T787', 'drug', '14879', '14882', '美金刚']
['T788', 'medical_indicator', '14918', '14

...

['T1109', 'pathological_mechanism', '19838', '19845', '脂质过氧化反应']
['T1110', 'pathological_mechanism', '19848', '19854', '神经元的损伤']
['T1111', 'disease', '19869', '19875', '阿尔茨海默病']
['T1112', 'medical_indicator', '19880', '19886', '学习记忆能力']
['T1115', 'medical_indicator', '19949', '19953', '认知功能']
['T1116', 'diagnostic_technology', '19983', '19996', '乳酸脱氢酶(LDH)分析法']
['T1117', 'medical_indicator', '19998', '20002', '细胞活性']
['T1118', 'inspection_method', '20004', '20011', '免疫荧光染色法']
['T1119', 'medical_indicator', '20013', '20017', '细胞性质']
['T1121', 'drug', '20052', '20054', 'CL']
['T1122', 'medical_indicator', '20060', '20066', '神经祖细胞数']
['T1123', 'medical_indicator', '20067', '20074', 'MAP2a/b']
['T1124', 'medical_indicator', '20075', '20084', 'SynapsinⅠ']
['T1125', 'drug', '20091', '20093', 'CL']
['T1126', 'medical_indicator', '20095', '20100', '神经祖细胞']
['T1127', 'disease', '20116', '20122', '阿尔茨海默病']
['T1128', 'medical_indicator', '20122', '20126', '认知功能']
['T1129', 'disease', '20405', '20

['T1', 'disease', '6', '12', '阿尔茨海默病']
['T4', 'disease', '384', '390', '阿尔茨海默病']
['T5', 'cause_of_disease', '391', '395', '颅脑外伤']
['T6', 'disease', '666', '672', '阿尔茨海默病']
['T7', 'disease', '745', '750', '老年痴呆症']
['T8', 'disease', '778', '784', '阿尔茨海默病']
['T9', 'disease', '841', '847', '阿尔茨海默病']
['T11', 'disease', '1000', '1006', '阿尔茨海默病']
['T12', 'symptom', '1020', '1024', '认知障碍']
['T13', 'disease', '1110', '1118', '轻度认知功能障碍']
['T14', 'disease', '1119', '1125', '阿尔茨海默病']
['T15', 'diagnostic_technology', '1139', '1144', '放射免疫法']
['T21', 'disease', '1749', '1755', '阿尔茨海默病']
['T22', 'medical_indicator', '1765', '1768', '葡萄糖']
['T23', 'medical_indicator', '1769', '1773', '真胰岛素']
['T24', 'disease', '2733', '2739', '阿尔茨海默病']
['T27', 'disease', '2098', '2104', '阿尔茨海默病']
['T28', 'cause_of_disease', '2849', '2855', 'β淀粉样蛋白']
['T29', 'disease', '2897', '2903', '阿尔茨海默病']
['T30', 'disease', '3187', '3193', '阿尔茨海默病']
['T31', 'symptom', '3240', '3244', '记忆障碍']
['T32', 'symptom', '3245', '3252', '心理

['T1', 'disease', '5', '11', '阿尔茨海默病']
['T3', 'inspection_method', '173', '178', '危险分数表']
['T5', 'disease', '250', '254', '老年痴呆']
['T7', 'disease', '326', '332', '阿尔茨海默病']
['T8', 'disease', '335', '341', ',血管性痴呆']
['T9', 'disease', '345', '350', '混合性痴呆']
['T11', 'disease', '438', '443', '老年痴呆症']
['T12', 'nursing_technology', '453', '457', '健康教育']
['T13', 'disease', '519', '523', '老年痴呆']
['T14', 'nursing_technology', '561', '565', '健康教育']
['T15', 'nursing_technology', '676', '680', '健康教育']
['T16', 'disease', '685', '690', '老年痴呆症']
['T17', 'diagnostic_technology', '821', '833', '苏木素-伊红(HE)染色']
['T18', 'inspection_method', '834', '840', 'TUNEL法']
['T19', 'inspection_method', '841', '848', '免疫组化染色法']
['T21', 'pathological_mechanism', '867', '872', '神经元凋亡']
['T22', 'disease', '748', '754', '阿尔茨海默病']
['T27', 'disease', '1098', '1104', '阿尔茨海默病']
['T29', 'nursing_technology', '1192', '1196', '心理护理']
['T30', 'nursing_technology', '1197', '1203', '家庭安全护理']
['T31', 'nursing_technology', '1204', '

['T232', 'medical_indicator', '2956', '2960', 'HAMD']
['T230', 'drug', '706', '709', '银杏叶']
['T233', 'drug', '713', '716', 'EGB']
['T130', 'drug', '4010', '4013', '石菖蒲']
['T139', 'drug', '4013', '4016', '远志散']
['T156', 'drug', '4502', '4505', '安理申']
['T234', 'drug', '4506', '4512', '盐酸多奈哌齐']
['T160', 'medical_indicator', '917', '923', '学习尝试次数']
['T235', 'medical_indicator', '928', '934', '记忆正确次数']
['T125', 'symptom', '3799', '3806', '行为和精神症状']
['T127', 'symptom', '3824', '3831', '行为和精神症状']
['T236', 'symptom', '3869', '3876', '行为和精神症状']
['T138', 'symptom', '7732', '7736', '精神障碍']
['T2', 'diagnostic_technology', '1840', '1843', 'MRI']
['T60', 'diagnostic_technology', '2201', '2204', 'MRI']
['T63', 'diagnostic_technology', '2355', '2357', 'MR']
['T77', 'cause_of_disease', '2325', '2332', 'β-淀粉样蛋白']
['T79', 'diagnostic_technology', '2333', '2335', 'MR']
['T86', 'diagnostic_technology', '2529', '2532', 'MRI']
['T186', 'diagnostic_technology', '2595', '2597', 'MR']
['T102', 'diagnostic_techn

开始读取文件:abstract_003-new.txt
开始写入文本datas/new/train.txt
开始读取文件:abstract_008-new.txt
开始写入文本datas/new/train.txt
